- Spain: La razón, 2o minutos
- Mexico: el universo, 
- Colombia: elheraldo, El Colombiano
- Argentina: la nacion, pagina/12
- Chile: diario de concepcion, biobiochile

In [233]:
import requests
from bs4 import BeautifulSoup
import time
import random
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import pandas as pd

# Spain

## El País

In [8]:
url = 'https://www.larazon.es'
response = requests.get(url)

print(f"Status code: {response.status_code}")


Status code: 200


In [9]:
links = ['https://www.larazon.es/internacional/', 'https://www.larazon.es/sociedad/', 'https://www.larazon.es/espana/', 
         'https://www.larazon.es/economia/','https://www.larazon.es/cultura/', 'https://elpais.com/salud/', 'https://elpais.com/gente/' ]

In [56]:
session = requests.Session()
retries = Retry(total=5, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
session.mount('https://', HTTPAdapter(max_retries=retries))

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def get_list_from_tema(url):
    try:
        response = session.get(url, headers=headers, timeout=10)
        article_links = []

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            title_elements = soup.find_all(class_='article__title')

            for element in title_elements:
                a_tag = element.find('a', href=True)
                if a_tag:
                    href = a_tag['href'].strip()
                    if href.startswith('/') and '/autor/' not in href.lower():
                        article_links.append(f"https://www.larazon.es{href}")
                    elif href.startswith('http') and '/autor/' not in href.lower():
                        article_links.append(href)

            # Filter out unwanted links and remove duplicates
            article_links = list(set(
                filter(lambda h: h.startswith('http') and not h.startswith('https://promociones.larazon.es/'), article_links)
            ))
        else:
            print(f"❌ Failed to fetch page: {url} (Status code: {response.status_code})")
        return article_links
    except Exception as e:
        print(f"⚠️ Exception in get_list_from_tema: {e}")
        return []


def get_article_text(link):
    try:
        art_resp = session.get(link, headers=headers, timeout=10)
        if art_resp.status_code == 200:
            art_soup = BeautifulSoup(art_resp.text, 'html.parser')
            content_div = art_soup.select_one('div.article-main__content')
            if content_div:
                paragraphs = content_div.find_all('p')
                return "\n".join(p.get_text(strip=True) for p in paragraphs)
    except Exception as e:
        print(f"⚠️ Exception in get_article_text ({link}): {e}")
    return None



In [57]:
data = []
for tema in links:
    urls = get_list_from_tema(tema)
    for article in urls:
        data.append(get_article_text(article))
        time.sleep(random.uniform(3,5))
    print(f'finished: {tema}')
    

finished: https://www.larazon.es/internacional/
finished: https://www.larazon.es/sociedad/
finished: https://www.larazon.es/espana/
finished: https://www.larazon.es/economia/
finished: https://www.larazon.es/cultura/
❌ Failed to fetch page: https://elpais.com/salud/ (Status code: 404)
finished: https://elpais.com/salud/
finished: https://elpais.com/gente/


In [61]:
df = pd.DataFrame({
    'TEXTO': data,
    'PAÍS': ['Spain']*len(data),
    'FUENTE': ['LaRazon']*len(data)
})


In [64]:
df.to_csv('datos_la_razon.csv', index = False)

## 20 Minutos

In [65]:
url = 'https://www.20minutos.es'
response = requests.get(url)

print(f"Status code: {response.status_code}")

Status code: 200


In [66]:
links = ['https://www.20minutos.es/nacional/', 'https://www.20minutos.es/internacional/', 'https://www.20minutos.es/lainformacion/', 
         'https://www.20minutos.es/deportes/','https://www.20minutos.es/gente/' ]

In [67]:
session = requests.Session()
retries = Retry(total=5, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
session.mount('https://', HTTPAdapter(max_retries=retries))

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
def get_list_from_tema(url):
    try:
        article_links = []

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            title_elements = soup.find_all(class_='media-content')
        
            for element in title_elements:
                a_tag = element.find('a', href=True)
                if a_tag:
                    href = a_tag['href'].strip()
                    article_links.append(href)
        else:
            print(f"❌ Failed to fetch page: {url} (Status code: {response.status_code})")
        return article_links
    except Exception as e:
        print(f"⚠️ Exception in get_list_from_tema: {e}")
        return []

def get_article_text(link):
    try:
        art_resp = session.get(link, headers=headers, timeout=10)
        if art_resp.status_code == 200:
            art_soup = BeautifulSoup(art_resp.text, 'html.parser')
            
            # Find all <p> tags with class 'paragraph' across the entire page
            paragraphs = art_soup.find_all('p', class_='paragraph')
            return " ".join(p.get_text(strip=True) for p in paragraphs)
    except Exception as e:
        print(f"⚠️ Exception in get_article_text ({link}): {e}")
    return None

In [68]:
data = []
for tema in links:
    urls = get_list_from_tema(tema)
    for article in urls:
        data.append(get_article_text(article))
        time.sleep(random.uniform(1,2))
    print(f'finished: {tema}')

finished: https://www.20minutos.es/nacional/
finished: https://www.20minutos.es/internacional/
finished: https://www.20minutos.es/lainformacion/
finished: https://www.20minutos.es/deportes/
finished: https://www.20minutos.es/gente/


In [70]:
df = pd.DataFrame({
    'TEXTO': data,
    'PAÍS': ['Spain']*len(data),
    'FUENTE': ['20minutos']*len(data)
})


In [76]:
df = df[df['TEXTO'].str.strip() != '']

df.to_csv('20mins.csv', index = False)

# Mexico

## el universal

In [89]:
links = ['https://www.eluniversal.com.mx/nacion', 'https://www.eluniversal.com.mx/mundo', 'https://www.eluniversal.com.mx/cartera', 
         'https://www.eluniversal.com.mx/deportes/', 'https://www.eluniversal.com.mx/cultura/']

session = requests.Session()
retries = Retry(total=5, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
session.mount('https://', HTTPAdapter(max_retries=retries))

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
def get_list_from_tema(url):
    try:
        article_links = []
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Find all <a> tags with a unique identifier in the class
            a_tags = soup.find_all('a', class_='cards-story-opener-fr')
            
            for a in a_tags:
                href = a.get('href')
                if href:
                    # If href is relative, you might want to add the domain
                    if href.startswith('/'):
                        href = 'https://www.eluniversal.com.mx' + href
                    article_links.append(href)
            return article_links
        else:
            print(f"❌ Failed to fetch: {url} (Status: {response.status_code})")
            return article_links
    except Exception as e:
        print(f"⚠️ Exception in get_list_from_tema: {e}")
        return []


def get_article_text(link):
    try:
        art_resp = session.get(link, headers=headers, timeout=10)
        if art_resp.status_code == 200:
            art_soup = BeautifulSoup(art_resp.text, 'html.parser')
                
                # Find all <p> tags with class 'paragraph' across the entire page
            paragraphs = art_soup.find_all('p', class_='sc__font-paragraph')
            joined_paragraph = ' '.join(p.get_text(strip=True) for p in paragraphs)       
        return joined_paragraph
    except Exception as e:
        print(f"⚠️ Exception in get_article_text ({link}): {e}")
    return None

In [78]:
data = []
for tema in links:
    urls = get_list_from_tema(tema)
    for article in urls:
        data.append(get_article_text(article))
        time.sleep(random.uniform(1,2))
    print(f'finished: {tema}')

finished: https://www.eluniversal.com.mx/nacion
finished: https://www.eluniversal.com.mx/mundo
finished: https://www.eluniversal.com.mx/cartera
finished: https://www.eluniversal.com.mx/deportes/
finished: https://www.eluniversal.com.mx/cultura/


In [82]:
df = pd.DataFrame({
    'TEXTO': data,
    'PAÍS': ['Mexico']*len(data),
    'FUENTE': ['ElUniversal']*len(data)
})

In [83]:
df.to_csv('datos_el_universo.csv', index = False)

## La Jornada

In [84]:
url = 'https://www.jornada.com.mx'
response = requests.get(url)

print(f"Status code: {response.status_code}")

Status code: 200


In [95]:
links = ['https://www.jornada.com.mx/categoria/politica', 'https://www.jornada.com.mx/categoria/economia', 'https://www.jornada.com.mx/categoria/mundo', 
         'https://www.jornada.com.mx/categoria/estados', 'https://www.jornada.com.mx/categoria/cultura', 'https://www.jornada.com.mx/categoria/deportes', 'https://www.jornada.com.mx/categoria/ciencia-y-tecnologia']
session = requests.Session()
retries = Retry(total=5, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
session.mount('https://', HTTPAdapter(max_retries=retries))

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
def get_list_from_tema(url):
    try:
        article_links = []
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Find all <h3> tags with the specified class
            h3_tags = soup.find_all('h3', class_='nota-titulo mb-1')
            
            for h3 in h3_tags:
                a_tag = h3.find('a')  # Find the <a> inside the <h3>
                if a_tag and a_tag.get('href'):
                    href = a_tag['href']
                    article_links.append(href)
            return article_links
        else:
            print(f"❌ Failed to fetch: {url} (Status: {response.status_code})")
            return article_links
    except Exception as e:
        print(f"⚠️ Exception in get_list_from_tema: {e}")
        return []


def get_article_text(link):
    try:
        art_resp = session.get(link, headers=headers, timeout=10)
        if art_resp.status_code == 200:
            art_soup = BeautifulSoup(art_resp.text, 'html.parser')
                
                # Find all <p> tags with class 'paragraph' across the entire page
            paragraphs = art_soup.find_all('p', class_='nota-justify')
            joined_paragraph = ' '.join(p.get_text(strip=True) for p in paragraphs)       
        return joined_paragraph
    except Exception as e:
        print(f"⚠️ Exception in get_article_text ({link}): {e}")
    return None

In [96]:
data = []
for tema in links:
    urls = get_list_from_tema(tema)
    for article in urls:
        data.append(get_article_text(article))
        time.sleep(random.uniform(1,2))
    print(f'finished: {tema}')

finished: https://www.jornada.com.mx/categoria/politica
finished: https://www.jornada.com.mx/categoria/economia
finished: https://www.jornada.com.mx/categoria/mundo
finished: https://www.jornada.com.mx/categoria/estados
finished: https://www.jornada.com.mx/categoria/cultura
finished: https://www.jornada.com.mx/categoria/deportes
finished: https://www.jornada.com.mx/categoria/ciencia-y-tecnologia


In [97]:
df = pd.DataFrame({
    'TEXTO': data,
    'PAÍS': ['Mexico']*len(data),
    'FUENTE': ['LaJornada']*len(data)
})

In [99]:
df.to_csv('datos_LaJornada.csv', index = False)

# Colombia

In [102]:
url = 'https://www.elheraldo.co'
response = requests.get(url)

print(f"Status code: {response.status_code}")

Status code: 200


In [120]:
links = ['https://www.elheraldo.co/atlantico/', 'https://www.elheraldo.co/deportes/', 'https://www.elheraldo.co/judicial/', 
         'https://www.elheraldo.co/economia/', 'https://www.elheraldo.co/region/']
session = requests.Session()
retries = Retry(total=5, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
session.mount('https://', HTTPAdapter(max_retries=retries))

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def get_list_from_tema(url):
    try:
        article_links = []
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Find all <h3> tags with the specified class
            h3_tags = soup.find_all(
                'h3', 
                class_='ingl-heading elheraldo__StyledHeadline-sc-1hpknk8-4 kUEUzH ingl-story-card__headline'
            )
            
            for h3 in h3_tags:
                a_tag = h3.find('a')
                if a_tag and a_tag.get('href'):
                    href = a_tag['href']
                    full_url = href if href.startswith('http') else urljoin('https://www.elheraldo.co', href)
                    article_links.append(full_url)
            return article_links
        else:
            print(f"❌ Failed to fetch: {url} (Status: {response.status_code})")
            return []
    except Exception as e:
        print(f"⚠️ Exception in get_list_from_tema: {e}")
        return []



def get_article_text(link):
    try:
        art_resp = session.get(link, headers=headers, timeout=10)
        if art_resp.status_code == 200:
            art_soup = BeautifulSoup(art_resp.text, 'html.parser')
                
                # Find all <p> tags with class 'paragraph' across the entire page
            paragraphs = art_soup.find_all('p', class_='elheraldo__StyledParagraph-sc-60z8lc-1 dEjMiD article-body__paragraph')
            joined_paragraph = ' '.join(p.get_text(strip=True) for p in paragraphs)       
        return joined_paragraph
    except Exception as e:
        print(f"⚠️ Exception in get_article_text ({link}): {e}")
    return None

In [116]:
len(data)

123

In [121]:
data = []
for tema in links:
    urls = get_list_from_tema(tema)
    for article in urls:
        data.append(get_article_text(article))
        time.sleep(random.uniform(1,2))
    print(f'finished: {tema}')

finished: https://www.elheraldo.co/atlantico/
finished: https://www.elheraldo.co/deportes/
finished: https://www.elheraldo.co/judicial/
finished: https://www.elheraldo.co/economia/
finished: https://www.elheraldo.co/region/


In [124]:
df = pd.DataFrame({
    'TEXTO': data,
    'PAÍS': ['Colombia']*len(data),
    'FUENTE': ['ElHeraldo']*len(data)
})

In [125]:
df.to_csv('datos_ElHeraldo.csv', index = False)

## El Colombiano (FALTA)

In [126]:
url = 'https://www.elcolombiano.com/espana'
response = requests.get(url)

print(f"Status code: {response.status_code}")

Status code: 200


In [128]:
links = ['https://www.elcolombiano.com/espana', 'https://www.elheraldo.co/deportes/', 'https://www.elheraldo.co/judicial/', 
         'https://www.elheraldo.co/economia/', 'https://www.elheraldo.co/region/']
session = requests.Session()
retries = Retry(total=5, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
session.mount('https://', HTTPAdapter(max_retries=retries))

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def get_list_from_tema(url):
    try:
        article_links = []
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Find all <h3> tags with the specified class
            h3_tags = soup.find_all(
                'h3', 
                class_='ingl-heading elheraldo__StyledHeadline-sc-1hpknk8-4 kUEUzH ingl-story-card__headline'
            )
            
            for h3 in h3_tags:
                a_tag = h3.find('a')
                if a_tag and a_tag.get('href'):
                    href = a_tag['href']
                    full_url = href if href.startswith('http') else urljoin('https://www.elheraldo.co', href)
                    article_links.append(full_url)
            return article_links
        else:
            print(f"❌ Failed to fetch: {url} (Status: {response.status_code})")
            return []
    except Exception as e:
        print(f"⚠️ Exception in get_list_from_tema: {e}")
        return []



def get_article_text(link):
    try:
        art_resp = session.get(link, headers=headers, timeout=10)
        if art_resp.status_code == 200:
            art_soup = BeautifulSoup(art_resp.text, 'html.parser')
                
                # Find all <p> tags with class 'paragraph' across the entire page
            paragraphs = art_soup.find_all('p', class_='elheraldo__StyledParagraph-sc-60z8lc-1 dEjMiD article-body__paragraph')
            joined_paragraph = ' '.join(p.get_text(strip=True) for p in paragraphs)       
        return joined_paragraph
    except Exception as e:
        print(f"⚠️ Exception in get_article_text ({link}): {e}")
    return None

In [134]:
article_links = []
response = requests.get(links[0])
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find all <h3> tags with the specified class
    h3_tags = soup.find_all(
        'h4', 
        class_='categorie__noticia__metadato'
    )
    print(h3_tags)
    for h3 in h3_tags:
        a_tag = h3.find('a')
        if a_tag and a_tag.get('href'):
            href = a_tag['href']
            full_url = href if href.startswith('http') else urljoin('https://www.elcolombiano.com/', href)
            
            article_links.append(full_url)

[<h4 class="categorie__noticia__metadato"> <span style="background:#006638"></span> <a href="/deportes/futbol">FÃºtbol</a> </h4>]


In [133]:
article_links

['https://www.elcolombiano.com/deportes/futbol']

# Argentina

## La Nación

In [136]:
url = 'https://www.lanacion.com.ar'
response = requests.get(url)

print(f"Status code: {response.status_code}")

Status code: 200


In [169]:
links = ['https://www.lanacion.com.ar/politica/', 'https://www.lanacion.com.ar/economia/', 'https://www.lanacion.com.ar/sociedad/', 
         'https://www.lanacion.com.ar/deportes/', 'https://www.lanacion.com.ar/lifestyle/']
session = requests.Session()
retries = Retry(total=5, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
session.mount('https://', HTTPAdapter(max_retries=retries))

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def get_list_from_tema(url):
    try:
        article_links = []
        response = requests.get(url)  # Assuming 'links' is a list and links[0] is a valid URL
        tema = url.split('/')[-2]
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Find all <a> tags on the page
            a_tags = soup.find_all('a')
            
            for a_tag in a_tags:
                href = a_tag.get('href')
                if href:
                    full_url = href if href.startswith('http') else urljoin(links[0], href)
                    article_links.append(full_url)
            return list(set([x for x in article_links if tema in x ]))
        else:
            print(f"❌ Failed to fetch: {url} (Status: {response.status_code})")
            return []
    except Exception as e:
        print(f"⚠️ Exception in get_list_from_tema: {e}")
        return []



def get_article_text(link):
    try:
        art_resp = session.get(link, headers=headers, timeout=10)
        if art_resp.status_code == 200:
            art_soup = BeautifulSoup(art_resp.text, 'html.parser')
                
                # Find all <p> tags with class 'paragraph' across the entire page
            paragraphs = art_soup.find_all('p', class_='com-paragraph --s')
            joined_paragraph = ' '.join(p.get_text(strip=True) for p in paragraphs)       
        return joined_paragraph
    except Exception as e:
        print(f"⚠️ Exception in get_article_text ({link}): {e}")
    return None

In [170]:
data = []
for tema in links:
    urls = get_list_from_tema(tema)
    print(len(urls))
    for article in urls:
        data.append(get_article_text(article))
        time.sleep(random.uniform(1,2))
    print(f'finished: {tema}')

33
finished: https://www.lanacion.com.ar/politica/
58
finished: https://www.lanacion.com.ar/economia/
30
finished: https://www.lanacion.com.ar/sociedad/
41
finished: https://www.lanacion.com.ar/deportes/
30
finished: https://www.lanacion.com.ar/lifestyle/


In [171]:
df = pd.DataFrame({
    'TEXTO': data,
    'PAÍS': ['Argentina']*len(data),
    'FUENTE': ['LaNación']*len(data)
})

In [173]:
df.to_csv('datos_LaNacion.csv', index = False)

## Pagina 12

In [174]:
url = 'https://www.pagina12.com.ar'
response = requests.get(url)

print(f"Status code: {response.status_code}")

Status code: 200


In [224]:
links = ['https://www.pagina12.com.ar/secciones/el-pais', 'https://www.pagina12.com.ar/secciones/economia', 'https://www.pagina12.com.ar/secciones/sociedad', 
         'https://www.pagina12.com.ar/deportes', 'https://www.pagina12.com.ar/secciones/el-mundo', 'https://www.pagina12.com.ar/secciones/ciencia']
session = requests.Session()
retries = Retry(total=5, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
session.mount('https://', HTTPAdapter(max_retries=retries))

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def get_list_from_tema(url):
    try:
        article_links = []
        response = requests.get(url)  # Assuming 'links' is a list and links[0] is a valid URL
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
        
            for a_tag in soup.find_all('a', href=True):
                href = a_tag['href']
                full_url = href if href.startswith('http') else urljoin(links[0], href)
        
                # Keep only article URLs that match the pattern: domain + numeric ID + hyphen
                if re.match(r'https://www\.pagina12\.com\.ar/\d+-', full_url):
                    article_links.append(full_url)

            return list(set(article_links))
        else:
            print(f"❌ Failed to fetch: {url} (Status: {response.status_code})")
            return []
    except Exception as e:
        print(f"⚠️ Exception in get_list_from_tema: {e}")
        return []



def get_article_text(link):
    try:
        art_resp = session.get(link, headers=headers, timeout=10)

        if art_resp.status_code == 200:
            soup = BeautifulSoup(art_resp.text, 'html.parser')  # Parse the response
            article_container = soup.find("div", class_="article-main-content")
            
            if article_container:
                paragraphs = article_container.find_all("p")
                article_text = "\n\n".join(p.get_text(strip=True) for p in paragraphs)      
        return article_text
    except Exception as e:
        print(f"⚠️ Exception in get_article_text ({link}): {e}")
    return None

In [225]:
data = []
for tema in links:
    urls = get_list_from_tema(tema)
    print(len(urls))
    for article in urls:
        data.append(get_article_text(article))
        time.sleep(random.uniform(1,2))
    print(f'finished: {tema}')

11
finished: https://www.pagina12.com.ar/secciones/el-pais
11
finished: https://www.pagina12.com.ar/secciones/economia
11
finished: https://www.pagina12.com.ar/secciones/sociedad
26
finished: https://www.pagina12.com.ar/deportes
11
⚠️ Exception in get_article_text (https://www.pagina12.com.ar/839607-lula-empezara-a-mover-la-agenda-externa-del-mercosur-con-ind): cannot access local variable 'article_text' where it is not associated with a value
⚠️ Exception in get_article_text (https://www.pagina12.com.ar/839617-jennifer-geerlings-simons-sera-la-primera-mujer-presidenta-d): cannot access local variable 'article_text' where it is not associated with a value
finished: https://www.pagina12.com.ar/secciones/el-mundo
11
⚠️ Exception in get_article_text (https://www.pagina12.com.ar/591520-especialistas-argentinos-crean-un-robot-capaz-de-descubrir-f): cannot access local variable 'article_text' where it is not associated with a value
⚠️ Exception in get_article_text (https://www.pagina12.com.a

In [226]:
df = pd.DataFrame({
    'TEXTO': data,
    'PAÍS': ['Argentina']*len(data),
    'FUENTE': ['Pagina12']*len(data)
})

In [228]:
df.to_csv('datos_Pagina12.csv', index = False)

# Chile

## Diario de Concepción

In [286]:
url = 'https://www.diarioconcepcion.cl'
response = requests.get(url)

print(f"Status code: {response.status_code}")

Status code: 200


In [287]:
import requests
from bs4 import BeautifulSoup

base_url = ["https://www.diarioconcepcion.cl/search?category=editorial&page={}", "https://www.diarioconcepcion.cl/search?category=opinion&page={}", 
           "https://www.diarioconcepcion.cl/search?category=ciudad&page={}", "https://www.diarioconcepcion.cl/search?category=politica&page={}", "https://www.diarioconcepcion.cl/search?category=economia&page={}"
           , "https://www.diarioconcepcion.cl/search?category=deportes&page={}", "https://www.diarioconcepcion.cl/search?category=cultura&page={}"]
all_links = set()
for url in base_url:
    x = url.split('=')[1]
    x = x.split('&')[0]
    for page in range(1, 20):
        print(f"Processing page {page}...")
        response = requests.get(url.format(page))
        soup = BeautifulSoup(response.text, "html.parser")
        
        for a in soup.find_all("a", href=True):
            href = a["href"]
            if x in href and href.endswith(".html"):
                if not href.startswith("http"):
                    href = "https://www.diarioconcepcion.cl" + href
                all_links.add(href)

# Convert to sorted list
final_links = sorted(all_links)




Processing page 1...
Processing page 2...
Processing page 3...
Processing page 4...
Processing page 5...
Processing page 6...
Processing page 7...
Processing page 8...
Processing page 9...
Processing page 10...
Processing page 11...
Processing page 12...
Processing page 13...
Processing page 14...
Processing page 15...
Processing page 16...
Processing page 17...
Processing page 18...
Processing page 19...
Processing page 1...
Processing page 2...
Processing page 3...
Processing page 4...
Processing page 5...
Processing page 6...
Processing page 7...
Processing page 8...
Processing page 9...
Processing page 10...
Processing page 11...
Processing page 12...
Processing page 13...
Processing page 14...
Processing page 15...
Processing page 16...
Processing page 17...
Processing page 18...
Processing page 19...
Processing page 1...
Processing page 2...
Processing page 3...
Processing page 4...
Processing page 5...
Processing page 6...
Processing page 7...
Processing page 8...
Processing pag

In [288]:
def get_news(link):
    response = requests.get(link)
    soup = BeautifulSoup(response.content, "html.parser")
    paragraphs = soup.find_all("p")
    all_text = "  ".join(p.get_text(strip=True) for p in paragraphs[1:-7])
    return all_text

In [289]:
data = []
for i in final_links:
    data.append(get_news(i))

In [290]:
df = pd.DataFrame({
    'TEXTO': data,
    'PAÍS': ['Chile']*len(data),
    'FUENTE': ['DiarioConcepción']*len(data)
})

In [292]:
df.to_csv('datos_DiaioConcepcion.csv', index = False)